In [ ]:
import random
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from stellargraph import StellarGraph

plt.rcParams['figure.constrained_layout.use'] = True
# Unzipping staller graph
# try:
#     with zipfile.ZipFile('dataset/Cora.zip') as archive:
#         archive.extractall('dataset/')
# except zipfile.BadZipFile as error:
#     print(error)

# dataset creation for the homogeneous graph embedding

# general properties of the node structure
NODE_UNIQUENESS_DEFINITION = {'A', 'B', 'C', 'D', 'E', 'F', 'G'}
NODE_SIZE = 2000

# generating unique node's ids according to given size
from_node_ids = np.random.choice(np.arange(0, NODE_SIZE), replace=False, size=NODE_SIZE)
assert len(np.unique(from_node_ids)) == len(from_node_ids)

# generate corrosponding edge values
def generate_corresponding_edges(fn):
    assert fn != None
    while True:
        tn = random.randint(0, NODE_SIZE)
        if fn != tn:
            return tn

# generate edge list of tuples
to_node_ids = np.fromiter(map(generate_corresponding_edges, from_node_ids), int)
assert len(to_node_ids) == len(from_node_ids)

square_edges = pd.DataFrame({
    'source': from_node_ids,
    'target': to_node_ids
})
square = StellarGraph(edges=square_edges)

def save_visual_graph(graph, filename):
    fig, ax = plt.subplots(num=None, figsize=(20, 20), dpi=60)
    plt.axis('off')
    pos = nx.spring_layout(graph)
    nx.draw_networkx_nodes(graph, pos, node_size=20, node_color='red')
    nx.draw_networkx_edges(graph, pos)
    nx.draw_networkx_labels(graph,pos)
    plt.savefig(filename,bbox_inches="tight")
    del fig

square_nx = square.to_networkx()
save_visual_graph(square_nx, 'custom-homogenous-graph.png');